# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 1:53PM,237883,12,TWL8053105,Twinlab Consolidated Corporation,Released
1,May 23 2022 1:53PM,237891,19,ACG-MP05232022,ACG North America LLC,Released
2,May 23 2022 1:51PM,237890,20,SSN-2173612658,Sartorius Stedim North America,Released
3,May 23 2022 1:47PM,237886,10,0085694806,ISDIN Corporation,Released
4,May 23 2022 1:47PM,237886,10,0085695075,ISDIN Corporation,Released
5,May 23 2022 1:47PM,237886,10,0085695111,ISDIN Corporation,Released
6,May 23 2022 1:47PM,237886,10,0085695119,ISDIN Corporation,Released
7,May 23 2022 1:47PM,237886,10,0085695121,ISDIN Corporation,Released
8,May 23 2022 1:47PM,237886,10,0085695120,ISDIN Corporation,Released
9,May 23 2022 1:47PM,237886,10,0085695132,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,237886,Released,15
43,237887,Released,1
44,237888,Released,14
45,237890,Released,1
46,237891,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237886,NaN,NaN,15.0
237887,NaN,NaN,1.0
237888,NaN,NaN,14.0
237890,NaN,NaN,1.0
237891,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237794,0.0,16.0,10.0
237795,0.0,0.0,1.0
237804,11.0,6.0,2.0
237808,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237794,0,16,10
237795,0,0,1
237804,11,6,2
237808,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237794,0,16,10
2,237795,0,0,1
3,237804,11,6,2
4,237808,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237794,,16,10
2,237795,,,1
3,237804,11,6,2
4,237808,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation
1,May 23 2022 1:53PM,237891,19,ACG North America LLC
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation
18,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd"
32,May 23 2022 1:44PM,237887,16,Sartorius Stedim Filters Inc.
33,May 23 2022 1:41PM,237885,19,"GUSA Granules USA, Inc."
35,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC"
36,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
42,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,,,1
1,May 23 2022 1:53PM,237891,19,ACG North America LLC,,,1
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America,,,1
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation,,,15
4,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd",,,14
5,May 23 2022 1:44PM,237887,16,Sartorius Stedim Filters Inc.,,,1
6,May 23 2022 1:41PM,237885,19,"GUSA Granules USA, Inc.",,,2
7,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",,,1
8,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,2,4
9,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,1,,
1,May 23 2022 1:53PM,237891,19,ACG North America LLC,1,,
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America,1,,
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation,15,,
4,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd",14,,
5,May 23 2022 1:44PM,237887,16,Sartorius Stedim Filters Inc.,1,,
6,May 23 2022 1:41PM,237885,19,"GUSA Granules USA, Inc.",2,,
7,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",1,,
8,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,2,
9,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,1,,
1,May 23 2022 1:53PM,237891,19,ACG North America LLC,1,,
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America,1,,
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation,15,,
4,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,1,NaN,NaN
1,May 23 2022 1:53PM,237891,19,ACG North America LLC,1,NaN,NaN
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America,1,NaN,NaN
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation,15,NaN,NaN
4,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd",14,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:53PM,237883,12,Twinlab Consolidated Corporation,1,0.0,0.0
1,May 23 2022 1:53PM,237891,19,ACG North America LLC,1,0.0,0.0
2,May 23 2022 1:51PM,237890,20,Sartorius Stedim North America,1,0.0,0.0
3,May 23 2022 1:47PM,237886,10,ISDIN Corporation,15,0.0,0.0
4,May 23 2022 1:46PM,237888,19,"AdvaGen Pharma, Ltd",14,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2854172,238,62.0,11.0
12,713583,3,1.0,0.0
15,237794,10,16.0,0.0
16,475771,2,0.0,0.0
18,1426745,6,0.0,0.0
19,1427294,29,2.0,0.0
20,1189331,80,9.0,14.0
22,475733,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2854172,238,62.0,11.0
1,12,713583,3,1.0,0.0
2,15,237794,10,16.0,0.0
3,16,475771,2,0.0,0.0
4,18,1426745,6,0.0,0.0
5,19,1427294,29,2.0,0.0
6,20,1189331,80,9.0,14.0
7,22,475733,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,238,62.0,11.0
1,12,3,1.0,0.0
2,15,10,16.0,0.0
3,16,2,0.0,0.0
4,18,6,0.0,0.0
5,19,29,2.0,0.0
6,20,80,9.0,14.0
7,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,238.0
1,12,Released,3.0
2,15,Released,10.0
3,16,Released,2.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
Executing,62.0,1.0,16.0,0.0,0.0,2.0,9.0,0.0
Released,238.0,3.0,10.0,2.0,6.0,29.0,80.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
1,Executing,62.0,1.0,16.0,0.0,0.0,2.0,9.0,0.0
2,Released,238.0,3.0,10.0,2.0,6.0,29.0,80.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
1,Executing,62.0,1.0,16.0,0.0,0.0,2.0,9.0,0.0
2,Released,238.0,3.0,10.0,2.0,6.0,29.0,80.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()